## Loading the Atoll Data

In [ ]:
# Pre-trained Model
!pip install git+https://github.com/karolzak/keras-unet

# Add other pip installs

from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imageio as iio

In [ ]:
# Constants to load data into the model
SIZE_X = 256 
SIZE_Y = 256
n_classes=3

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Rotating all the Images
import os
import shutil
from PIL import Image 
import PIL 
 

# assign directory
# Note: create a JP_CV_Notebooks shortcut to "My Drive" before running
GDRIVE_IMGDIR = os.getenv('GDRIVE_IMG')
GDRIVE_MASKDIR = os.getenv('GDRIVE_MASK')

ImageDirectory = GDRIVE_IMGDIR
MaskDirectory = GDRIVE_MASKDIR

train_images = []
train_masks = [] 
# iterate over files 
plt.figure(figsize=(12, 8))
   

for images in os.listdir(ImageDirectory):
    for masks in os.listdir(MaskDirectory):
        f = os.path.join(ImageDirectory, images)
        b = os.path.join(MaskDirectory, masks)
        
        if(images == masks):
         
            img = cv2.imread(f, 0)    
            #img = iio.imread(f)  
            img = cv2.resize(img, (SIZE_Y, SIZE_X))
            train_images.append(img)

            hi = cv2.imread(b)       
            #mask = iio.imread(b)
            mask = hi[:,:,2]
            mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST) 
            train_masks.append(mask)
           # print(images + "=" + masks)

           # plt.subplot(231)
           # plt.title(images)
           # plt.imshow(img)
           # plt.subplot(232)
            #plt.title(masks)
           # plt.imshow(mask)
            #plt.show()

            
            break

train_images = np.array(train_images)
train_masks = np.array(train_masks) 

In [ ]:
#Image Processing

plt.imshow(train_masks[3])
plt.show()
train_masks[train_masks == 0] = 51 #Replacing 0 class to 51
plt.imshow(train_masks[3])
plt.show()

print(np.unique(train_masks)) # There should be 3 classes printed. If there is four, manually replace the 4th class. 
print(train_masks.shape)

In [ ]:
#Run to check if all images match with masks. Don't need to call every time.

for n in range(316):
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image' + str(n))
    plt.imshow(train_images[n])
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(train_masks[n])
    plt.show()

### Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)


In [ ]:
train_images = np.expand_dims(train_images, axis=3)
train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

### Data Splitting into Train and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

#Further split training data t a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.2, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))



test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
np.reshape(X_test, (32, 256, 256))

### Calculate Class Weights(if relevant):

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = "balanced",
                                                 classes = np.unique(train_masks_reshaped_encoded),
                                                 y = train_masks_reshaped_encoded)
print("Class weights are...:", class_weights)
class_weight_dict = dict(enumerate(class_weights))


class_weighting = {i : class_weights[i] for i in range(3)}
print(class_weight_dict)
print(class_weighting)

class_weights = np.zeros((227,256,256))
class_weights[:, 0] += 0.4368813386093144 # Change value if what is printed by class_weighting is different
class_weights[:, 1] += 4.84032092458291 #  Change value if what is printed by class_weighting is different
class_weights[:, 2] += 1.9823527953404594 # Change value if what is printed by class_weighting is different







## Models

### Model 1:

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

def multi_unet_model(n_classes=3, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
    # Building the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    # Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

### Model 2:

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda


################################################################
def bn_conv_relu(input, filters, bachnorm_momentum, **conv2d_args):
    x = BatchNormalization(momentum=bachnorm_momentum)(input)
    x = Conv2D(filters, **conv2d_args)(x)
    return x

def bn_upconv_relu(input, filters, bachnorm_momentum, **conv2d_trans_args):
    x = BatchNormalization(momentum=bachnorm_momentum)(input)
    x = Conv2DTranspose(filters, **conv2d_trans_args)(x)
    return x

def satellite_unet(
    input_shape,
    num_classes=3,
    output_activation='sigmoid',
    num_layers=1):

    inputs = Input(input_shape)   
    
    filters = 64
    upconv_filters = 96

    kernel_size = (3,3)
    activation = 'relu'
    strides = (1,1)
    padding = 'same'
    kernel_initializer = 'he_normal'

    conv2d_args = {
        'kernel_size':kernel_size,
        'activation':activation, 
        'strides':strides,
        'padding':padding,
        'kernel_initializer':kernel_initializer
        }

    conv2d_trans_args = {
        'kernel_size':kernel_size,
        'activation':activation, 
        'strides':(2,2),
        'padding':padding,
        'output_padding':(1,1)
        }

    bachnorm_momentum = 0.01

    pool_size = (2,2)
    pool_strides = (2,2)
    pool_padding = 'valid'

    maxpool2d_args = {
        'pool_size':pool_size,
        'strides':pool_strides,
        'padding':pool_padding,
        }
    
    x = Conv2D(filters, **conv2d_args)(inputs)
    c1 = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)    
    x = bn_conv_relu(c1, filters, bachnorm_momentum, **conv2d_args)
    x = MaxPooling2D(**maxpool2d_args)(x)

    down_layers = []

    for l in range(num_layers):
        x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
        x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
        down_layers.append(x)
        x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
        x = MaxPooling2D(**maxpool2d_args)(x)

    x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
    x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
    x = bn_upconv_relu(x, filters, bachnorm_momentum, **conv2d_trans_args)

    for conv in reversed(down_layers):        
        x = concatenate([x, conv])  
        x = bn_conv_relu(x, upconv_filters, bachnorm_momentum, **conv2d_args)
        x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
        x = bn_upconv_relu(x, filters, bachnorm_momentum, **conv2d_trans_args)

    x = concatenate([x, c1])
    x = bn_conv_relu(x, upconv_filters, bachnorm_momentum, **conv2d_args)
    x = bn_conv_relu(x, filters, bachnorm_momentum, **conv2d_args)
           
    outputs = Conv2D(num_classes, kernel_size=(1,1), strides=(1,1), activation=output_activation, padding='valid') (x)       
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

    

## Training

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]
import tensorflow as tf

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS) #For MOdel 1
  #  return satellite_unet(input_shape=(256,256,1)) #For Model 2

model = get_model()
model.compile(optimizer='adam', loss= tf.keras.losses.BinaryFocalCrossentropy(gamma=2), metrics=['acc'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'], sample_weight_mode='temporal') #Uncomment this if using class weights
model.summary()

### Fitting

In [ ]:
history = model.fit(X_train, y_train_cat, 
                    batch_size = 8, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                   # sample_weight=class_weights, #If using class weights, uncomment this
                    shuffle=False)
                    

DRIVE_FILESAVE = os.getenv('DRIVE_FILESAVE')
model.save(DRIVE_FILESAVE)

In [ ]:
# # In case you need to load the model
from tensorflow import keras
MODEL_DIR = os.getenv('MODEL_1')
model = keras.models.load_model(MODEL_DIR)

## Evaluation


### Evaluating the Model

In [ ]:
_, acc = model.evaluate(X_test, y_test_cat)
model.evaluate(X_test, y_test_cat)
#model.summary()
print("Accuracy is = ", (acc * 100.0), "%")

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
y_pred=model.predict(X_test)

print(y_test_cat.shape)
print(y_pred.shape)

#IMPORTANT STEP: if anything about the image size or number of images is changed, make sure to change reshaped value.
#Should go from: (Number of images, SIZE_X, SIZE_Y, 3) to (Number of images * SIZE_X * SIZE_Y, 3)
Y_testing = np.reshape(y_test_cat, (2097152, 3))
Y_predict = np.reshape(y_pred, (2097152, 3))

print(Y_testing.shape)
print(Y_predict.shape)

Y_predict=np.argmax(Y_predict, axis=1)
Y_testing=np.argmax(Y_testing, axis=1)

print(Y_testing.shape)
print(Y_predict.shape)


result =confusion_matrix(Y_testing, Y_predict)
disp = ConfusionMatrixDisplay(confusion_matrix=result)
disp.plot()
plt.show()
print(result)

### Mean IoU

In [ ]:
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)

# Using built in keras function
from keras.metrics import MeanIoU
n_classes = 3
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


# To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[1,0]+ values[2,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[0,1]+ values[2,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[0,2]+ values[1,2])
# class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
# print("IoU for class4 is: ", class4_IoU)

plt.imshow(train_images[2, :,:,0], cmap='gray')
plt.imshow(train_masks[2], cmap='gray')

### Comparing Ground Truth to Prediction

In [ ]:
import random
test_img_number = random.randint(0, len(X_test))
for x in range(len(X_test)):
    test_img = X_test[x]
    ground_truth=y_test[x]
    test_img_norm=test_img[:,:,0][:,:,None]
    test_img_input=np.expand_dims(test_img_norm, 0)
    prediction = (model.predict(test_img_input))
    predicted_img=np.argmax(prediction, axis=3)[0,:,:]


    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:,:,0], cmap='gray')
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(ground_truth[:,:,0], cmap='jet')
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(predicted_img, cmap='jet')
    plt.show()

## Experimentation

**Table for Model 1**

BFC = BinaryFocalCrossentropy

| Model | Epochs| Batchsize|Loss Function|Class_Weight|Accuracy|Mean IOU|
|:------|-------|----------|-------------|------------|--------|--------|
|1_model| 30    |  8       |  BFC        |  No        |  79    |   0.26 |
|2_model|100    |  8       |  BFC        | Yes        |83.9    |   0.42 |
|3_model|200    |  8       |  BFC        | Yes        | 85     |   0.56 |
|4_model| 50    |  16      |  BFC        |  Yes       | 79     | 0.36   |
|5_model| 200   |  8       |  BFC        |   No       |88.9    |   0.61 |
|6_model| 30    |  8       |  BFC        | Yes        | 80     |  0.35  |


## Observations from the results

### Interrater Agreement

In [ ]:
from sklearn.metrics import cohen_kappa_score
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
def image_to_a_numpy_array(fname):

  img = np.array(Image.open(fname))

  height, width, num_bands = img.shape

  print('Image Size: ', img.shape)

  res = img.reshape(-1, num_bands)

  res = res.astype('float64')

  return res

In [ ]:
def calculate_and_plot_cohen_kappa(file_name, folder_paths):

  img_1_path = folder_paths[0] + file_name
  img_2_path = folder_paths[1] + file_name
  img_3_path = folder_paths[2] + file_name

  img_1 = image_to_a_numpy_array(img_1_path)
  img_2 = image_to_a_numpy_array(img_2_path)
  img_3 = image_to_a_numpy_array(img_3_path)

  imgs = [img_1, img_2, img_3]

  ck_12 = cohen_kappa_score(img_1.flatten(), img_2.flatten())
  ck_13 = cohen_kappa_score(img_1.flatten(), img_3.flatten())
  ck_23 = cohen_kappa_score(img_2.flatten(), img_3.flatten())

  print('------------------------------------------------------')
  print(f'Printing Cohen Kappa Scores for image {file_name}')
  print('------------------------------------------------------')
  print('Cohen Kappa Score of Labeler 1 and Labeler 2: ', ck_12)
  print('------------------------------------------------------')
  print('Cohen Kappa Score of Labeler 1 and Labeler 3: ', ck_13)
  print('------------------------------------------------------')
  print('Cohen Kappa Score of Labeler 2 and Labeler 3: ', ck_23)
  print('------------------------------------------------------\n')
  print('Plotting Graphs...\n')
  plt.figure(figsize=(10, 10))

  plt.subplot(211)

  bar_chart = plt.bar(['L_1 - L_2', 'L_1 - L_3', 'L_2 - L_3'], 
                    [ck_12, ck_13, ck_23], align='center')

  plt.ylim(0, 1)  
  plt.title(f'Cohen Kappa Score for {file_name}')
  plt.xlabel('Pairs of Labelers')
  plt.ylabel('Cohen Kappa Score')

  plt.subplot(212)

  data = {'Labeler 1': [ck_12, ck_13],
        'Labeler 2': [ck_12, ck_23],
        'Labeler 3': [ck_13, ck_23]}

  df = pd.DataFrame(data)
  heatmap = sns.heatmap(df, annot=True, cmap='coolwarm')
  plt.xlabel('Labelers')
  plt.ylabel('Other Labelers')

  return [ck_12, ck_13, ck_23]

In [ ]:
AGREEMENT = os.getenv('AGREEMENT_FILEPATH')
folder_path = AGREEMENT
folder_paths = [folder_path+os.getenv('NAME_1'), folder_path+os.getenv('NAME_2'), folder_path+os.getenv('NAME_3')]

img_names = ['1.png', '2.png']

length_of_test_dataset = len(img_names)

results = []

for img_name in img_names:

  res = calculate_and_plot_cohen_kappa(img_name, folder_paths)

  results.append(res)


In [ ]:
ck_12_average_score = 0
ck_13_average_score = 0
ck_23_average_score = 0

for i in range(len(img_names)):

  ck_12_average_score += results[i][0]/length_of_test_dataset
  ck_13_average_score += results[i][1]/length_of_test_dataset
  ck_23_average_score += results[i][2]/length_of_test_dataset

print('------------------------------------------------------')
print('Printing Average Cohen Kappa Scores')
print('------------------------------------------------------')
print('Average Cohen Kappa Score of Labeler 1 and Labeler 2: ', ck_12_average_score)
print('------------------------------------------------------')
print('Average Cohen Kappa Score of Labeler 1 and Labeler 3: ', ck_13_average_score)
print('------------------------------------------------------')
print('Average Cohen Kappa Score of Labeler 2 and Labeler 3: ', ck_23_average_score)
print('------------------------------------------------------\n')
print('Plotting Graphs...\n')

plt.figure(figsize=(10, 10))

plt.subplot(211)

bar_chart = plt.bar(['L_1 - L_2', 'L_1 - L_3', 'L_2 - L_3'], 
                    [ck_12_average_score, ck_13_average_score, ck_23_average_score], align='center')

plt.ylim(0, 1)  
plt.title('Average Cohen Kappa Coefficient')
plt.xlabel('Pairs of Labelers')
plt.ylabel('Cohen Kappa Coefficient')
plt.savefig('Cohen Kappa Score.png')

plt.subplot(212)

data = {'Labeler 1': [ck_12_average_score, ck_13_average_score],
        'Labeler 2': [ck_12_average_score, ck_23_average_score],
        'Labeler 3': [ck_13_average_score, ck_23_average_score]}

df = pd.DataFrame(data)
heatmap = sns.heatmap(df, annot=True, cmap='coolwarm')
plt.xlabel('Labelers')
plt.ylabel('Other Labelers')

## Future work plan for improvement

* Train on more data: more annotation with Dash Doodler
* Different approach to annotation: use Kmeans images as masks for annotation instead of dash doodler since dash doodler susceptible to human error. One caveat: classes are consistently labeled with Kmeans
* Different models: We use UNet here, but there are other segmentation models online that could be used
* More hyperparameter tuning could be done. Most of the tuning was not documented since results all ended up looking/averaging the same.
* Create a more streamlined way of data manipulation. Currently, going from Dash_Doodler to feeding the model takes far too many steps in manipulation: Much of it isn't documented here because its all grunt work that can be easily resolved by a python script. If interested in the process and script, refer to Miscellaneous.ipynb in folder.